In [2]:
import os
import json
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-05-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

# Create an assistant
assistant = client.beta.assistants.create(
    name="Data Prep",
    instructions="""
    As a Financial Analyst, you will leverage your expertise to generate tailored Financial Analysis Reports that cater to the specific requirements of clients. This role involves in-depth analysis of financial statements and market data to uncover insights into a company's financial performance and stability.
You will engage directly with clients to gather essential information and refine the report based on their feedback, ensuring that the final product precisely meets their needs and expectations.

Key Objectives:

Analytical Precision: Utilize analytical skills to interpret financial data, identify trends, and detect anomalies.
Effective Communication: Simplify and convey complex financial information in a clear and actionable manner for non-specialist audiences.
Client Focus: Tailor reports dynamically based on client feedback, aligning the analysis with their strategic goals.
Quality Assurance: Maintain the highest standards of quality and integrity in report generation, adhering to established benchmarks for analytical rigor.
Performance Indicators:

The effectiveness of the Financial Analysis Report is measured by its ability to provide actionable insights that support corporate decision-making, identify areas for operational improvement, and evaluate the company's financial health. Success is reflected in the report's contribution to informed investment decisions and strategic planning.
    """,
    tools=[{"type": "file_search"}],
    model= os.getenv("DEPLOYMENT_NAME"), #You must replace this value with the deployment name for your model.
    temperature=0
)

In [3]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_QQgblqWtqIM5eHeg89Cbts5Q",
  "created_at": 1724050763,
  "description": null,
  "instructions": "\n    As a Financial Analyst, you will leverage your expertise to generate tailored Financial Analysis Reports that cater to the specific requirements of clients. This role involves in-depth analysis of financial statements and market data to uncover insights into a company's financial performance and stability.\nYou will engage directly with clients to gather essential information and refine the report based on their feedback, ensuring that the final product precisely meets their needs and expectations.\n\nKey Objectives:\n\nAnalytical Precision: Utilize analytical skills to interpret financial data, identify trends, and detect anomalies.\nEffective Communication: Simplify and convey complex financial information in a clear and actionable manner for non-specialist audiences.\nClient Focus: Tailor reports dynamically based on client feedback, aligning the analysis with their

In [4]:
# Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_apwd80CExWzo5VxlBFREHo4Y', created_at=1724050967, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [5]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""The report will include the following sections and I want to extract the capital expenditure and other metric like 

Operating Activities "Change in Working Capital",
        "Net Cash from Operating Activities"
Investing Activities has "Acquisition of Fixed Assets & Intangibles",
        "Net Cash from Investing Activities"
Financing Activities includes "Dividends Paid",
        "Cash from (Repayment of) Debt",
        "Net Cash from Financing Activities",
Net Change includes  Net Change in Cash
Metadata includes "Report Date", "Publish Date", "Source"
Profitability Metrics like "EBITDA",
        "Gross Profit Margin",
        "Operating Margin",
        "Net Profit Margin",
        "Return on Equity",
        "Return on Assets",
        "Return On Invested Capital",
Liquidity Metrics include Current Ratio
Solvency Metrics like  "Total Debt", "Liabilities to Equity Ratio", "Debt Ratio",
Cash Flow Metrics like "Free Cash Flow", "Free Cash Flow to Net Income", "Cash Return On Invested Capital",
Other Important Metrics like "Piotroski F-Score", "Net Debt / EBITDA", "Dividend Payout Ratio
Capital Expenditure    
"""
)

In [6]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_VtcRFcQGbv52PfXr9jA739zd",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "The report will include the following sections and I want to extract the capital expenditure and other metric like \n\nOperating Activities \"Change in Working Capital\",\n        \"Net Cash from Operating Activities\"\nInvesting Activities has \"Acquisition of Fixed Assets & Intangibles\",\n        \"Net Cash from Investing Activities\"\nFinancing Activities includes \"Dividends Paid\",\n        \"Cash from (Repayment of) Debt\",\n        \"Net Cash from Financing Activities\",\nNet Change includes  Net Change in Cash\nMetadata includes \"Report Date\", \"Publish Date\", \"Source\"\nProfitability Metrics like \"EBITDA\",\n        \"Gross Profit Margin\",\n        \"Operating Margin\",\n        \"Net Profit Margin\",\n        \"Return on Equit

In [7]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [8]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

completed


In [9]:
# import time
# from IPython.display import clear_output

# start_time = time.time()

# status = run.status

# while status not in ["completed", "cancelled", "expired", "failed"]:
#     time.sleep(5)
#     run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
#     print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
#     status = run.status
#     print(f'Status: {status}')
#     clear_output(wait=True)

# messages = client.beta.threads.messages.list(
#   thread_id=thread.id
# ) 

# print(f'Status: {status}')
# print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
# print(messages.model_dump_json(indent=2))

In [10]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_aPZuoVRSo1Weq0W3q3CsLc0V",
      "assistant_id": "asst_QQgblqWtqIM5eHeg89Cbts5Q",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Apologies, but it seems that no documents have been uploaded yet. Could you please upload the relevant file so that I can assist you in extracting the capital expenditure information and other metrics?"
          },
          "type": "text"
        }
      ],
      "created_at": 1724051273,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_MU2hRnGjqQEpN2GpdX2VQrwk",
      "status": null,
      "thread_id": "thread_apwd80CExWzo5VxlBFREHo4Y"
    },
    {
      "id": "msg_VtcRFcQGbv52PfXr9jA739zd",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        

In [10]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)  # Outputs: assistant-1YGVTvNzc2JXajI5JU9F0HMD

assistant-kDuDAwS2bMPodkoMNXyZBx77


In [11]:
content = client.files.content(image_file_id)

image= content.write_to_file("sinewave.png")

In [12]:
# Add a new user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the code you used to generate the sinewave"
)

In [14]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions  but these will override the default instructions
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)



queued


In [16]:
status = run.status
print(status)

queued


In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))